In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

In [ ]:
DATA_PATH = "plan-velo-2026.geojson"

In [17]:
# Read the data (Limit data to Paris)
map_df = gpd.read_file(filename=DATA_PATH, bbox=(2.248651,48.812706,2.420266,48.905743))

In [32]:
# Drop useless columns
to_drop = ["id_local", "reseau_loc", "nom_loc", "num_iti", "code_com_g", "date_maj", "comm", "source", "project_c", "ref_geo",
"id_osm", "code_com_d", "regime_d", "regime_g", "largeur_d", "largeur_g", "access_ame", "d_service"]

for col in to_drop:
    if(col in map_df):
        map_df.drop(col, axis=1, inplace=True)

In [33]:
from shapely.geometry import mapping

def convert_line_to_points(row, id, skipped_ids=[]) -> pd.DataFrame:
    """
    Converts a line to a dataframe of points
    
    Args:
        row (pd.Series): The row to convert
        id (int): The id of the row
        skipped_ids (list): The list of ids to skip
    
    Returns:
        pd.DataFrame: The dataframe of points
    """
    path_id = []
    order = []
    longitude = []
    latitude = []

    line = mapping(row.geometry)["coordinates"]
    if(len(line) > 1):
        skipped_ids.append(id - 1)
    line = line[0]

    for point in line:
        point_long, point_lat = point
        longitude.append(point_long)
        latitude.append(point_lat)
        order.append(len(order) + 1)
        path_id.append(id)

    df = pd.DataFrame({
        "Path ID": path_id,
        "Order": order,
        "Longitude": longitude,
        "Latitude": latitude
    })
    for col in map_df.columns:
        if(col != "geometry"):
            df[col] = row[col]
    
    return df

In [34]:
# Prepare the data
to_export = pd.DataFrame()

skipped_lines = []

for (i,row) in enumerate(map_df.iterrows(), start=1):
    line_df = convert_line_to_points(row[1], i, skipped_ids=skipped_lines)
    to_export = pd.concat([to_export, line_df])

to_export

,Path ID,Order,Longitude,Latitude,reseau,statut
0,1.0,1.0,2.40622,48.835684,vélopolitain,existant
1,1.0,2.0,2.396393,48.839272,vélopolitain,existant
2,1.0,3.0,2.387638,48.841362,vélopolitain,existant
3,1.0,4.0,2.385964,48.842153,vélopolitain,existant
4,1.0,5.0,2.377338,48.846305,vélopolitain,existant
...,...,...,...,...,...,...
6,310.0,7.0,2.286476,48.863,secondaire,à réaliser
7,310.0,8.0,2.286508,48.862823,secondaire,à réaliser
8,310.0,9.0,2.28668,48.862633,secondaire,à réaliser
9,310.0,10.0,2.286819,48.862548,secondaire,à réaliser


In [ ]:
# Export
to_export.to_csv("amenagement.csv", sep=";")